In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, datasets, models
from retinaface import RetinaFace
import cv2
import numpy as np
from PIL import Image
import os
from glob import glob
from sklearn.model_selection import StratifiedShuffleSplit
from tqdm import tqdm
import pickle

# ============================================================================
# STEP 1: ONE-TIME PREPROCESSING - Run this ONCE and save results
# ============================================================================

def preprocess_and_save_faces(data_path, output_path, target_size=(48, 48), skip_detection=True):
    """
    Detect faces once and save cropped faces to disk.
    This should be run ONCE before training.
    
    Args:
        skip_detection: If True, skips face detection entirely (fastest, for pre-cropped datasets like FER2013).
    """
    os.makedirs(output_path, exist_ok=True)
    
    if skip_detection:
        print("✓ Skipping face detection (dataset already contains faces)")
        print("✓ This will take 2-3 minutes for the entire dataset\n")
    
    # Initialize face detector if needed
    face_cascade = None
    if not skip_detection:
        print("✓ Using OpenCV Haar Cascade for face detection")
        face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
    
    for split in ['train', 'test']:
        split_path = os.path.join(data_path, split)
        output_split_path = os.path.join(output_path, split)
        
        classes = os.listdir(split_path)
        
        for emotion_class in classes:
            class_path = os.path.join(split_path, emotion_class)
            output_class_path = os.path.join(output_split_path, emotion_class)
            os.makedirs(output_class_path, exist_ok=True)
            
            image_files = glob(os.path.join(class_path, "*.png"))
            
            print(f"Processing {split}/{emotion_class}: {len(image_files)} images")
            
            successful = 0
            for img_path in tqdm(image_files, desc=f"{split}/{emotion_class}"):
                try:
                    img = Image.open(img_path).convert("RGB")
                    img_np = np.array(img)
                    
                    if skip_detection:
                        # Just resize - no detection needed
                        crop = cv2.resize(img_np, target_size)
                    else:
                        # OpenCV Haar Cascade detection
                        gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
                        faces = face_cascade.detectMultiScale(
                            gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30)
                        )
                        
                        if len(faces) > 0:
                            # Use the largest face
                            x, y, w, h = max(faces, key=lambda f: f[2] * f[3])
                            crop = img_np[y:y+h, x:x+w]
                            crop = cv2.resize(crop, target_size)
                        else:
                            crop = cv2.resize(img_np, target_size)  # Fallback
                    
                    # Save cropped face
                    crop_pil = Image.fromarray(crop)
                    output_file = os.path.join(output_class_path, os.path.basename(img_path))
                    crop_pil.save(output_file)
                    successful += 1
                    
                except Exception as e:
                    pass  # Skip problematic images
            
            print(f"✓ Successfully processed {successful}/{len(image_files)} images\n")
    
    print("✓ Preprocessing complete! Cropped faces saved to:", output_path)


def extract_and_save_features(preprocessed_path, output_file, device='cpu'):
    """
    Extract features from all preprocessed faces ONCE and save to disk.
    This should be run ONCE after preprocessing faces.
    """
    # Load pre-trained feature extractor
    feature_extractor = models.efficientnet_b0(
        weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1
    )
    feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-1])  # Remove classifier
    feature_extractor.eval()
    feature_extractor.to(device)
    
    # Transformation for feature extraction
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # EfficientNet input size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    features_dict = {'train': [], 'test': []}
    
    for split in ['train', 'test']:
        dataset = datasets.ImageFolder(
            os.path.join(preprocessed_path, split),
            transform=transform
        )
        dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=4)
        
        split_features = []
        split_labels = []
        
        print(f"\nExtracting features for {split} set...")
        with torch.no_grad():
            for images, labels in tqdm(dataloader):
                images = images.to(device)
                features = feature_extractor(images)
                features = features.squeeze(-1).squeeze(-1)  # Remove spatial dimensions
                
                split_features.append(features.cpu())
                split_labels.append(labels)
        
        features_dict[split] = {
            'features': torch.cat(split_features),
            'labels': torch.cat(split_labels),
            'classes': dataset.classes
        }
    
    # Save features
    torch.save(features_dict, output_file)
    print(f"\nFeatures saved to: {output_file}")
    print(f"Feature dimension: {features_dict['train']['features'].shape[1]}")


# ============================================================================
# STEP 2: FAST TRAINING DATASET - Uses pre-extracted features
# ============================================================================

class PreExtractedFeatureDataset(Dataset):
    """
    Fast dataset that loads pre-extracted features from memory/disk.
    """
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


# ============================================================================
# STEP 3: TRAINING FUNCTIONS
# ============================================================================

class FER_EfficientNetClassifier(nn.Module):
    def __init__(self, feature_dim, num_classes=7):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(feature_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        return self.classifier(x)


def train_model(model, train_loader, val_loader, criterion, optimizer, 
                num_epochs, device='cpu', patience=5):
    """
    Fast training function with early stopping.
    """
    model.to(device)
    best_val_acc = 0.0
    patience_counter = 0
    
    for epoch in range(num_epochs):
        print(f"\n--- Epoch {epoch+1}/{num_epochs} ---")
        
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        train_bar = tqdm(train_loader, desc='Training')
        for features, labels in train_bar:
            features, labels = features.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * features.size(0)
            _, predicted = torch.max(outputs, 1)
            train_correct += (predicted == labels).sum().item()
            train_total += labels.size(0)
            
            train_bar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Acc': f'{train_correct/train_total:.4f}'
            })
        
        train_loss /= train_total
        train_acc = train_correct / train_total
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for features, labels in tqdm(val_loader, desc='Validation'):
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * features.size(0)
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == labels).sum().item()
                val_total += labels.size(0)
        
        val_loss /= val_total
        val_acc = val_correct / val_total
        
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%")
        
        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
            print(f"✓ New best model saved! (Val Acc: {val_acc*100:.2f}%)")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"\nEarly stopping triggered after {epoch+1} epochs")
                break
    
    return model


# ============================================================================
# STEP 4: MAIN TRAINING SCRIPT
# ============================================================================

def main():
    # Paths
    DATA_PATH = "C:/adam/AMIT_Diploma/grad_project/archive (1)"
    PREPROCESSED_PATH = "C:/adam/AMIT_Diploma/grad_project/preprocessed_faces"
    FEATURES_FILE = "C:/adam/AMIT_Diploma/grad_project/extracted_features.pt"
    
    # ========================================================================
    # OPTION A: First time setup (run once)
    # ========================================================================
    # FASTEST (5-8 minutes) - Skip face detection for FER2013
    print("Step 1: Preprocessing faces...")
    preprocess_and_save_faces(DATA_PATH, PREPROCESSED_PATH, skip_detection=True)
    # 
    print("\nStep 2: Extracting features...")
    extract_and_save_features(PREPROCESSED_PATH, FEATURES_FILE)
    
    # ========================================================================
    # OPTION B: Fast training (run every time after preprocessing)
    # ========================================================================
    print("Loading pre-extracted features...")
    features_dict = torch.load(FEATURES_FILE)
    
    # Get feature dimension
    feature_dim = features_dict['train']['features'].shape[1]
    print(f"Feature dimension: {feature_dim}")
    
    # Split train into train/val
    train_features = features_dict['train']['features']
    train_labels = features_dict['train']['labels']
    
    # Stratified split
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
    train_idx, val_idx = next(splitter.split(
        np.arange(len(train_labels)),
        train_labels.numpy()
    ))
    
    # Create datasets
    train_dataset = PreExtractedFeatureDataset(
        train_features[train_idx],
        train_labels[train_idx]
    )
    val_dataset = PreExtractedFeatureDataset(
        train_features[val_idx],
        train_labels[val_idx]
    )
    test_dataset = PreExtractedFeatureDataset(
        features_dict['test']['features'],
        features_dict['test']['labels']
    )
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    
    print(f"Train samples: {len(train_dataset)}")
    print(f"Val samples: {len(val_dataset)}")
    print(f"Test samples: {len(test_dataset)}")
    
    # Initialize model
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = FER_EfficientNetClassifier(feature_dim=feature_dim, num_classes=7)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Train
    print("\nStarting training...")
    trained_model = train_model(
        model, train_loader, val_loader, criterion, optimizer,
        num_epochs=50, device=device, patience=5
    )
    
    print("\nTraining complete!")


if __name__ == "__main__":
    main()

Step 1: Preprocessing faces...
✓ Skipping face detection (dataset already contains faces)
✓ This will take 2-3 minutes for the entire dataset

Processing train/angry: 3995 images


train/angry: 100%|██████████| 3995/3995 [00:02<00:00, 1530.10it/s]


✓ Successfully processed 3995/3995 images

Processing train/disgusted: 436 images


train/disgusted: 100%|██████████| 436/436 [00:00<00:00, 1618.93it/s]


✓ Successfully processed 436/436 images

Processing train/fearful: 4097 images


...

Train Loss: 0.3455, Train Acc: 87.36%
Val Loss: 1.7234, Val Acc: 59.74%

--- Epoch 23/50 ---


Validation: 100%|██████████| 45/45 [00:00<00:00, 1026.79it/s]


Train Loss: 0.3218, Train Acc: 88.26%
Val Loss: 1.7652, Val Acc: 57.65%

--- Epoch 24/50 ---


Validation: 100%|██████████| 45/45 [00:00<00:00, 817.39it/s]

Train Loss: 0.3121, Train Acc: 88.79%
Val Loss: 1.7921, Val Acc: 58.13%

Early stopping triggered after 24 epochs

Training complete!
